# ETL (Extract, Transform, Load)

El proceso ETL es fundamental para preparar los datos de Siniestros Viales para su análisis. Al extraer, transformar y cargar los datos de manera efectiva, se mejora la calidad de la información, se facilita la comprensión de los patrones y tendencias, y se posibilita la toma de decisiones informada en cuestiones de seguridad vial. La normalización, limpieza y organización adecuadas de los datos son esenciales para garantizar la validez y utilidad de los resultados del análisis.

### Se importan las librerías necesarias para el proceso de ETL

In [8]:
# Importar librerías
import pandas as pd # Importa la librería pandas para el manejo de datos
import calendar # Importa la librería calendar para trabajar con fechas
import locale # Importa la librería locale para configurar la configuración regional
import seaborn as sns # Importa la librería seaborn para la visualización de datos
import matplotlib.pyplot as plt # Importa la librería matplotlib.pyplot para la generación de gráficos
from datetime import datetime # Importa la clase datetime del módulo datetime para trabajar con fechas y horas
import warnings # Importa la función filterwarnings del módulo warnings para ignorar las advertencias
warnings.filterwarnings("ignore")
import utils # Script con funciones de ETL y EDA
import utilsGemini 
import utilsGPT

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Extracción de los datos desde un archivo xlsx, que contiene varias hojas (sheets), las de interés para el proyecto son HECHOS y VICTIMAS respectivamente. Se hará uso de algunas funciones de ETL y EDA recopiladas en el Script `utils.py`

In [4]:
hechos = pd.read_excel("data\homicidios.xlsx", sheet_name="HECHOS")
victimas = pd.read_excel("data\homicidios.xlsx", sheet_name="VICTIMAS")

### Exploración Dataframe `hechos`

In [5]:
hechos.head()  # Visualizamos los primeros 5 registros

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


Verificamos el tipo de datos, y el total de nulos por columna en porcentaje.

In [14]:
utils.verificar_tipo_datos_y_nulos(hechos)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
11,Altura,[<class 'float'>],18.53,81.47,567
12,Cruce,"[<class 'str'>, <class 'float'>]",75.43,24.57,171
13,Dirección Normalizada,"[<class 'str'>, <class 'float'>]",98.85,1.15,8
10,Calle,"[<class 'str'>, <class 'float'>]",99.86,0.14,1
19,VICTIMA,[<class 'str'>],100.00,0.00,0
18,PARTICIPANTES,[<class 'str'>],100.00,0.00,0
17,pos y,[<class 'str'>],100.00,0.00,0
16,pos x,[<class 'str'>],100.00,0.00,0
15,XY (CABA),[<class 'str'>],100.00,0.00,0
14,COMUNA,[<class 'int'>],100.00,0.00,0


Observaciones: 

* Es necesario renombrar columnas para manejar un solo formato de nombres de columnas.
* 